In [28]:
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
from scipy import poly1d
from datetime import datetime
import yfinance as yf
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')
plt.rcParams['figure.figsize'] = (10,7)

In [8]:
cobrandingPairsDf = pd.read_csv('cobrandingPairs.csv')

In [9]:
cobrandingPairsDf = cobrandingPairsDf.drop('Unnamed: 0',1)

In [10]:
cobrandingPairsDf

,Stock1,Stock2
0,NVDA,TSM
1,BMWYY,LVMUY
2,NKE,AAPL
3,NFLX,AMZN
4,MSFT,AMZN
5,FB,AMZN
6,MSFT,EBAY
7,FB,AMZN
8,FB,MSFT
9,QCOM,TSM


In [17]:
dfSP = pd.read_csv('sp_data.csv')
dfSP = dfSP[:-1]

In [18]:
dfSP = dfSP.drop('Unnamed: 0',1)
dfSP = dfSP.drop('Unnamed: 506',1)

In [19]:
dfSP

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,WYNN,XEL,XLNX,XYL,YUM,ZBRA,ZBH,ZION,ZTS,Date
0,166.347397,73.888474,70.061684,260.269989,169.237976,46.289951,276.769989,27.090000,157.008972,15.022331,...,125.465141,54.252647,113.940872,76.228203,97.850586,195.889999,120.165443,45.380920,101.123032,2019-05-10 00:00:00
1,163.264282,74.082359,69.989822,261.940002,169.861115,46.111988,275.790009,27.209999,158.266037,15.153941,...,117.718361,54.793182,107.820885,74.882660,96.866585,181.009995,116.904419,43.123123,100.193748,2019-05-13 00:00:00
2,163.804535,74.111435,69.576584,258.920013,169.705307,45.499008,278.480011,27.959999,156.514053,15.276150,...,119.118965,54.347469,110.296173,73.624863,97.127052,186.720001,116.993073,43.507229,100.094894,2019-05-14 00:00:00
3,162.137253,73.190506,69.055550,252.369995,166.511795,42.918560,267.700012,26.240000,153.791946,15.144540,...,120.983215,54.262131,111.899628,74.541389,97.223923,185.179993,116.796043,42.554863,100.678154,2019-05-15 00:00:00
4,163.199097,73.704277,70.465942,256.690002,169.033508,44.193954,271.859985,27.320000,154.168106,15.116338,...,122.973030,54.745762,103.765022,74.356125,97.717987,190.020004,116.333000,42.969749,101.755714,2019-05-16 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,198.610001,119.529999,114.680000,308.640015,292.170013,91.150002,503.459991,78.550003,201.860001,27.350000,...,127.809998,70.750000,122.029999,117.360001,121.089996,482.420013,173.550003,58.220001,173.860001,2021-05-05 00:00:00
501,199.380005,117.540001,113.900002,305.000000,291.600006,88.690002,490.700012,78.610001,204.149994,26.709999,...,122.779999,71.550003,122.139999,118.750000,121.290001,471.410004,173.229996,59.180000,167.520004,2021-05-06 00:00:00
502,201.440002,118.510002,115.779999,300.790009,289.519989,90.080002,486.690002,77.830002,202.809998,26.469999,...,126.290001,71.320000,124.430000,119.339996,121.230003,488.579987,172.490005,59.270000,171.550003,2021-05-07 00:00:00
503,202.410004,118.790001,116.080002,295.720001,291.160004,93.029999,483.609985,77.889999,204.160004,25.450001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-07 00:00:00


In [75]:
##Arg: dataframe of stock pairs, 
##Returns: return after trading in those positions 
def getTradingSignals(df):
    returns = []
    removeIndex = []
    for index,stocks in df.iterrows():
        stock1 = stocks['Stock1']
        stock2 = stocks['Stock2']
        kalmanDf = pd.DataFrame()

        pair1 = yf.Ticker(stock1)
        pair2 = yf.Ticker(stock2)

        pair1 = pair1.history(period = '2y')
        pair2 = pair2.history(period = '2y')
        if(pair1.shape != pair2.shape):
            removeIndex.append(index)
            continue

        kalmanDf[stock1] = pair1['Close'].values.astype(float)
        kalmanDf[stock2] = pair2['Close'].values.astype(float)
        kalmanDf['ratio'] = kalmanDf[stock1]/kalmanDf[stock2] 
        kf = KalmanFilter(transition_matrices = [1],
                  observation_matrices = [1],
                  initial_state_mean = 0,
                  initial_state_covariance = 1,
                  observation_covariance=1,
                  transition_covariance=.0001)
        mean, cov = kf.filter(kalmanDf['ratio'].values)
        kalmanDf['mean'] = mean.squeeze()
        kalmanDf['cov'] = cov.squeeze()
        kalmanDf['std'] = np.sqrt(kalmanDf['cov'])
        kalmanDf = kalmanDf.dropna()
        kalmanDf['ma'] = kalmanDf['ratio'].rolling(5).mean()
        kalmanDf['z_score'] = (kalmanDf['ma'] - kalmanDf['mean'])/kalmanDf['std']

        # Initialise positions as zero
        kalmanDf['position_1'] = np.nan
        kalmanDf['position_2'] = np.nan

        # Generate buy, sell and square off signals as: z<-1 buy, z>1 sell and -1<z<1 liquidate the position
        for i in range (kalmanDf.shape[0]):
            if kalmanDf['z_score'].iloc[i] < -1:
                kalmanDf['position_1'].iloc[i] = 1
                kalmanDf['position_2'].iloc[i] = -round(kalmanDf['ratio'].iloc[i],0)
            if kalmanDf['z_score'].iloc[i] > 1:
                kalmanDf['position_1'].iloc[i] = -1
                kalmanDf['position_2'].iloc[i] = round(kalmanDf['ratio'].iloc[i],0)
            if (abs(kalmanDf['z_score'].iloc[i]) < 1) & (abs(kalmanDf['z_score'].iloc[i]) > 0):
                kalmanDf['position_1'].iloc[i] = 0
                kalmanDf['position_2'].iloc[i] = 0
        # Calculate returns
        
        kalmanDf['returns'] = ((kalmanDf[stock1]-kalmanDf[stock1].shift(1))/kalmanDf[stock1].shift(1))*kalmanDf['position_1'].shift(1)+ ((kalmanDf[stock2]-kalmanDf[stock2].shift(1))/kalmanDf[stock2].shift(1))*kalmanDf['position_2'].shift(1)
        kalmanDf['returns'].sum()
        kalmanDf.to_csv("signals/"+stock1+"_"+stock2+".csv")
        returns.append(kalmanDf['returns'].sum())
    return returns, removeIndex


In [77]:
returns, removeIndex = getTradingSignals(cobrandingPairsDf)
print(len(returns))
cobrandingPairsDf['returns'] = returns

13


In [78]:
cobrandingPairsDf

,Stock1,Stock2,returns
0,NVDA,TSM,2.833159
1,BMWYY,LVMUY,0.000000
2,NKE,AAPL,0.071972
3,NFLX,AMZN,0.000000
4,MSFT,AMZN,0.000000
5,FB,AMZN,0.000000
6,MSFT,EBAY,2.405793
7,FB,AMZN,0.000000
8,FB,MSFT,0.005091
9,QCOM,TSM,0.096405


In [65]:
kmeansCointPairsDf = pd.read_csv('finalPairs.csv')
kmeansCointPairsDf = kmeansCointPairsDf.drop('Unnamed: 0',1)

In [66]:
kmeansCointPairsDf

,Stock1,Stock2
0,ABMD,ILMN
1,ACN,BLK
2,ACN,CMI
3,ACN,INTU
4,ACN,ISRG
...,...,...
5228,WAB,ZION
5229,WMT,ZTS
5230,WM,WRK
5231,WY,ZBH


In [67]:
returns,removeIndex = getTradingSignals(kmeansCointPairsDf)
print(len(returns))
# kmeansCointPairsDf['returns'] = returns

5037


In [ ]:
kmeansCointPairsDf

In [79]:
kmeansCointCorrPairsDf = pd.read_csv('corrCointPairs.csv')
kmeansCointCorrPairsDf = kmeansCointCorrPairsDf.drop('Unnamed: 0',1)

In [81]:
returns, removeIndex = getTradingSignals(kmeansCointCorrPairsDf)
print(len(returns))

1331


In [82]:
len(kmeansCointCorrPairsDf)

1344

In [83]:
kmeansCointCorrPairsDf = kmeansCointCorrPairsDf.drop(removeIndex)

In [85]:
kmeansCointCorrPairsDf['returns'] = returns

In [93]:
kmeansCointCorrPairsDf.sort_values(by = 'returns', ascending = False)

,Stock1,Stock2,returns
1060,IPGP,PENN,12.677960
804,FRT,NOV,9.341557
1066,IQV,LB,9.288114
449,BXP,NOV,9.026585
980,IEX,LB,8.593600
...,...,...,...
396,ADP,RF,-4.698861
255,AXP,IVZ,-4.803362
94,INTU,TSLA,-4.868321
773,EOG,MRO,-5.426012


In [94]:
###Merge the dataframes

In [95]:
###Get the top 10 pairs and their returns
###Visualizations for these stocks

In [96]:
####Calculate the sharpe ratio